线程是操作系统进行运算调度的最小单位。他被包含在进程之中，是进程中的实际运作单位。一个线程指的是进程中一个单一顺序的控制流，一个进程中可以并发多个线程，每个线程并行执行不同的任务，当中每一个线程共享当前进程的资源。

Python由于存在全局解释器锁，任意时刻都只有一个线程在运行代码，致使多线程不能充分利用计算机多核的特性。
- 程序是CPU密集型的(大数据计算)，使用Python的多线程无法提升程序的效率；
- 程序是IO密集型的(文件读写、网络数据传输)，Python多线程可以提高程序的整体效率。（因为在IO操作期间，线程会释放GIL，从而允许其他线程运行）。

In [1]:
from threading import Thread, currentThread
import time

def fun_thread(second, t_name):
    print('启动线程-->', currentThread().getName(), ': ', currentThread().is_alive())
    print('setName修改线程名称')
    currentThread().setName(t_name)
    time.sleep(30)
    print('{}线程结束'.format(currentThread().getName()))

In [4]:
threads = []
for i in range(3):
    t = Thread(target=fun_thread, name='thread-{}'.format(i), args=(5, 'my_{}_Thread)'.format(i)))
    threads.append(t)
    t.start()
    time.sleep(2)
for t in threads:
    t.join()

C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:5: DeprecationWarning: currentThread() is deprecated, use current_thread() instead
  print('启动线程-->', currentThread().getName(), ': ', currentThread().is_alive())
C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:5: DeprecationWarning: getName() is deprecated, get the name attribute instead
  print('启动线程-->', currentThread().getName(), ': ', currentThread().is_alive())
C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:7: DeprecationWarning: currentThread() is deprecated, use current_thread() instead
  currentThread().setName(t_name)
C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:7: DeprecationWarning: setName() is deprecated, set the name attribute instead
  currentThread().setName(t_name)


启动线程--> thread-0 :  True
setName修改线程名称
启动线程--> thread-1 :  True
setName修改线程名称
启动线程--> thread-2 :  True
setName修改线程名称


C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:9: DeprecationWarning: currentThread() is deprecated, use current_thread() instead
  print('{}线程结束'.format(currentThread().getName()))
C:\Users\wanghaibo\AppData\Local\Temp\ipykernel_4816\3666243994.py:9: DeprecationWarning: getName() is deprecated, get the name attribute instead
  print('{}线程结束'.format(currentThread().getName()))


my_0_Thread)线程结束
my_1_Thread)线程结束
my_2_Thread)线程结束


In [7]:
# 类继承 创建多线程

import threading
import time
import random
from queue import Queue

# 创建锁对象
counter_lock = threading.Lock()
print_lock = threading.Lock()

# 创建队列
task_queue = Queue()
result_queue = Queue()

# 共享资源
counter = 0

def worker_function(worker_id):
    """线程工作函数"""
    global counter
    
    with print_lock:
        print(f"👷 Worker {worker_id} 开始工作...")
    
    # 模拟工作
    time.sleep(random.uniform(0.5, 1.5))
    
    # 使用锁保护共享资源
    with counter_lock:
        counter += 1
        with print_lock:
            print(f"🔒 Worker {worker_id} 增加计数器: {counter}")
    
    # 处理队列任务
    while not task_queue.empty():
        task = task_queue.get()
        if task is None:
            break
        result = f"Worker {worker_id} 处理任务: {task} -> 结果: {task * 2}"
        result_queue.put(result)
        task_queue.task_done()
        time.sleep(0.3)
    
    with print_lock:
        print(f"✅ Worker {worker_id} 工作完成")

class WorkerThread(threading.Thread):
    """自定义线程类"""
    def __init__(self, worker_id):
        super().__init__()
        self.worker_id = worker_id
        self.daemon = True  # 设置为守护线程
    
    def run(self):
        """线程执行内容"""
        worker_function(self.worker_id)

def main():
    # 填充任务队列
    for i in range(1, 11):
        task_queue.put(i)
    
    # 创建线程
    threads = []
    for i in range(1, 6):
        thread = WorkerThread(i)
        threads.append(thread)
        thread.start()
    
    # 等待所有线程完成
    task_queue.join()
    
    # 显示结果
    print("\n" + "="*50)
    print("所有任务完成！")
    print(f"最终计数器值: {counter}")
    
    # 显示处理结果
    print("\n任务处理结果:")
    while not result_queue.empty():
        print(f"  - {result_queue.get()}")
    
    print("\n" + "="*50)
    print("多线程示例完成！")

if __name__ == "__main__":
    print("="*50)
    print("Python多线程编程示例")
    print("="*50)
    print("启动5个工作线程处理10个任务...\n")
    main()

Python多线程编程示例
启动5个工作线程处理10个任务...

👷 Worker 1 开始工作...
👷 Worker 2 开始工作...
👷 Worker 3 开始工作...
👷 Worker 4 开始工作...
👷 Worker 5 开始工作...
🔒 Worker 4 增加计数器: 1
🔒 Worker 1 增加计数器: 2
🔒 Worker 2 增加计数器: 3
🔒 Worker 3 增加计数器: 4
🔒 Worker 5 增加计数器: 5

所有任务完成！
最终计数器值: 5

任务处理结果:
  - Worker 4 处理任务: 1 -> 结果: 2
  - Worker 1 处理任务: 2 -> 结果: 4
  - Worker 2 处理任务: 3 -> 结果: 6
  - Worker 4 处理任务: 4 -> 结果: 8
  - Worker 1 处理任务: 5 -> 结果: 10
  - Worker 2 处理任务: 6 -> 结果: 12
  - Worker 4 处理任务: 7 -> 结果: 14
  - Worker 3 处理任务: 8 -> 结果: 16
  - Worker 5 处理任务: 9 -> 结果: 18
  - Worker 1 处理任务: 10 -> 结果: 20

多线程示例完成！
✅ Worker 2 工作完成
✅ Worker 3 工作完成
✅ Worker 5 工作完成
✅ Worker 4 工作完成
✅ Worker 1 工作完成


In [9]:
import threading
import time
import random

# 创建锁来保护共享资源和打印输出
print_lock = threading.Lock()

# 线程工作函数
def worker_task(worker_id, sleep_time):
    """线程执行的任务"""
    with print_lock:
        print(f"🚀 线程 {worker_id} 启动，将休眠 {sleep_time:.2f} 秒")
    
    # 模拟工作
    time.sleep(sleep_time)
    
    with print_lock:
        print(f"✅ 线程 {worker_id} 完成工作")
    
    # 返回结果
    return f"线程 {worker_id} 结果: 休眠了 {sleep_time:.2f} 秒"

def main():
    print("=" * 50)
    print("Thread类直接创建线程示例")
    print("=" * 50)
    print("创建5个线程，每个线程执行不同时间的任务...\n")
    
    # 创建并启动线程
    threads = []
    
    for i in range(5):
        # 生成随机休眠时间 (0.5-2.5秒)
        sleep_time = random.uniform(0.5, 2.5)
        
        # 创建线程，target指定工作函数，args传递参数
        thread = threading.Thread(target=worker_task, args=(i, sleep_time))
        
        # 启动线程
        thread.start()
        threads.append(thread)
    
    # 等待所有线程完成
    for thread in threads:
        thread.join()
    
    # 打印所有结果
    print("\n" + "=" * 50)
    print("所有线程已完成工作！")
    print("任务结果:")
    print("\n" + "=" * 50)
    print("示例完成！")

if __name__ == "__main__":
    main()

Thread类直接创建线程示例
创建5个线程，每个线程执行不同时间的任务...

🚀 线程 0 启动，将休眠 2.31 秒
🚀 线程 1 启动，将休眠 0.69 秒
🚀 线程 2 启动，将休眠 2.47 秒
🚀 线程 3 启动，将休眠 1.49 秒
🚀 线程 4 启动，将休眠 1.37 秒
✅ 线程 1 完成工作
✅ 线程 4 完成工作
✅ 线程 3 完成工作
✅ 线程 0 完成工作
✅ 线程 2 完成工作

所有线程已完成工作！
任务结果:

示例完成！
